# 🧹 Introduction to Data Cleaning: Retail Transactions Dataset

In the world of data analysis, raw data rarely arrives in perfect condition. This notebook demonstrates essential steps for **data cleaning** using a small but insightful sample of retail transaction data. 

The dataset includes the following fields:
- `Transaction ID`
- `Item`
- `Quantity`
- `Price Per Unit`
- `Total Spent`
- `Payment Method`
- `Location`
- `Transaction Date`

While these inconsistencies may seem minor, they can have a significant impact on downstream analysis and insights. Therefore, our objectives in this notebook are to:
- ✅ **Identify** errors and inconsistencies  
- 🛠️ **Correct or manage** the dirty data  
- 📦 **Prepare** a clean dataset for further exploration or modeling

Let’s roll up our sleeves and turn this messy data into something insightful!


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv


In [2]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Loading the Dataframe

In [3]:
df=pd.read_csv("/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv")

## Overview of the Dataset

In [4]:
print(df.shape)

(10000, 8)


In [5]:
df.head(20)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
None


In [7]:
print("\nNumber of duplicate rows:")
print(df.duplicated().sum())

print("\nRows with invalid 'Total Spent' values:")
print(df[~df["Total Spent"].apply(lambda x: str(x).replace('.', '', 1).isdigit())])

for col in ['Payment Method', 'Location','Item']:
    print(f"\nUnique values in {col}:")
    print(df[col].unique())


Number of duplicate rows:
0

Rows with invalid 'Total Spent' values:
     Transaction ID      Item Quantity Price Per Unit Total Spent  \
2       TXN_4271903    Cookie        4            1.0       ERROR   
25      TXN_7958992  Smoothie        3            4.0         NaN   
31      TXN_8927252   UNKNOWN        2            1.0       ERROR   
42      TXN_6650263       Tea        2            1.5     UNKNOWN   
65      TXN_4987129  Sandwich        3            NaN         NaN   
...             ...       ...      ...            ...         ...   
9893    TXN_3809533     Juice        2            NaN       ERROR   
9954    TXN_1191659    Coffee        4            2.0       ERROR   
9977    TXN_5548914     Juice        2            3.0       ERROR   
9988    TXN_9594133      Cake        5            3.0         NaN   
9993    TXN_4766549  Smoothie        2            4.0         NaN   

      Payment Method  Location Transaction Date  
2        Credit Card  In-store       2023-07-19  
2

## Issues found with the Dataset
### There are many missing values, Incorrect data types, Unique values in categorical columns, Invalid data, etc.

## Delete Unnecessary Rows
### Rows with 'Item' marked as Unknown, containing errors, or missing data have no values for the data analysis and Predictive Model Building

In [8]:
df = df[df['Item'] != 'UNKNOWN']
df = df[df['Item'] != 'ERROR']
df = df.dropna(subset=['Item'])

## Convert Datatypes

In [9]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

## Fill Missing Values

In [10]:

df['Payment Method'].fillna('Unknown', inplace=True)
df['Location'].fillna('Unknown', inplace=True)

df['Payment Method'] = df['Payment Method'].replace({'UNKNOWN': 'Unknown'})
df['Location'] = df['Location'].replace({'UNKNOWN': 'Unknown'})


df['Quantity'].fillna(df['Quantity'].mean(), inplace=True)

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
mean_date = df['Transaction Date'].mean()
df['Transaction Date'].fillna(mean_date, inplace=True)

df['Price Per Unit'] = df.groupby('Item')['Price Per Unit'].transform(
    lambda x: x.fillna(x.mean())
)

df['Total Spent'] = df['Quantity'] * df['Price Per Unit']

## Check Data Again

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9031 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    9031 non-null   object        
 1   Item              9031 non-null   object        
 2   Quantity          9031 non-null   float64       
 3   Price Per Unit    9031 non-null   float64       
 4   Total Spent       9031 non-null   float64       
 5   Payment Method    9031 non-null   object        
 6   Location          9031 non-null   object        
 7   Transaction Date  9031 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 635.0+ KB


In [12]:
for col in ['Payment Method', 'Location', 'Item']:
    print(f"\nUnique values in {col}:")
    print(df[col].unique())


Unique values in Payment Method:
['Credit Card' 'Cash' 'Unknown' 'Digital Wallet' 'ERROR']

Unique values in Location:
['Takeaway' 'In-store' 'Unknown' 'ERROR']

Unique values in Item:
['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']


## Drop rows where 'Payment Method' or 'Location' has 'ERROR'

In [13]:
df = df[(df['Payment Method'] != 'ERROR') & (df['Location'] != 'ERROR')]

## Final Check

In [14]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [15]:
df.dtypes

Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

In [16]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [17]:
df.duplicated().sum()

0

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8430 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    8430 non-null   object        
 1   Item              8430 non-null   object        
 2   Quantity          8430 non-null   float64       
 3   Price Per Unit    8430 non-null   float64       
 4   Total Spent       8430 non-null   float64       
 5   Payment Method    8430 non-null   object        
 6   Location          8430 non-null   object        
 7   Transaction Date  8430 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 592.7+ KB


## Export data to CSV format

In [19]:
df.to_csv("cleaned_data.csv", index=False)